In [4]:
#-------------------------------------------------------------------------
# Variational Quantum Classifier (VQC)
# Chapter 6 in the QUANTUM COMPUTING AND QUANTUM MACHINE LEARNING BOOK
# Code demonstrates applying the phase separation step for two qubits
#-------------------------------------------------------------------------
# Version 1.0
# (c) 2025 Jesse Van Griensven, Roydon Fraser, and Jose Rosas 
# Licence:  MIT - Citation of this work is required
#-------------------------------------------------------------------------
# Qiskit changes frequently. 
# We recommend using the latest version from the book code repository at:
# https://github.com/pedroer/quantum-computing-for-engineers/blob/main/requirements.txt
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.opflow import Z, StateFn, CircuitStateFn
from qiskit.algorithms.optimizers import COBYLA
import warnings
warnings.filterwarnings('ignore')
#-------------------------------------------------------------------------

def create_pqc(theta_val):
    """ Create a simple parameterized quantum circuit (PQC) """
    qc = QuantumCircuit(1)
    qc.rx(theta_val, 0)
    return qc

#-------------------------------------------------------------------------

def loss_function(theta_values, data, labels):
    """ Define the loss function """
    # Extract the single parameter value from the array
    theta_val = theta_values[0]
    qc = create_pqc(theta_val)
    results = []
    
    # Loop over each data point and perform feature encoding via an extra rotation.
    for x in data:
        qc_data = qc.copy()
        qc_data.rx(x, 0)  # Feature encoding
        # Build a StateFn for measurement using the Z operator.
        result = StateFn(Z, is_measurement=True) @ CircuitStateFn(qc_data)
        results.append(result.eval())
    
    # Convert the results to real predictions and compute the mean squared error.
    predictions = np.array([np.real(r) for r in results])
    return np.mean((predictions - labels) ** 2)
#-------------------------------------------------------------------------

# Generate synthetic data
data   = np.linspace(0, 2. * np.pi, 10)
labels = np.sin(data)

# Optimize the circuit parameters using COBYLA
optimizer = COBYLA(maxiter=100)
initial_theta  = np.array([0.1])
optimal_result = optimizer.minimize(
    fun=lambda theta: loss_function(theta, data, labels),
    x0=initial_theta
)

# The optimizer returns an object containing the solution in the 'x' attribute.
print(f"Optimal θ: {optimal_result.x}")


Optimal θ: [-1.57069844]
